# Übung: Rechnen mit Polynomen – Bausteine der Post-Quantum-Kryptographie

In dieser Übung lernst ihr:
- wie man Polynome in Python darstellt,
- wie man sie addiert und multipliziert,
- wie man modulare Polynom-Arithmetik durchführt (wie z. B. in Kyber)
- und wie diese Rechenart kryptographisch genutzt wird.


## Grundlegende Polynomrechnung

In [1]:
from sympy import symbols, Poly

# Definiere eine symbolische Variable 'x'
x = symbols('x')

# Erstelle zwei Polynome f(x) und g(x) mit ganzzahligen Koeffizienten
# Hinweis: domain='ZZ' ist implizit – alle Koeffizienten sind in den ganzen Zahlen
f = Poly(x**3 + 2*x + 1, x)
g = Poly(3*x**2 + x + 2, x)

# Berechne die Summe der beiden Polynome
print("f(x) + g(x) =", f + g)
# Berechne das Produkt der beiden Polynome
print("f(x) * g(x) =", f * g)


f(x) + g(x) = Poly(x**3 + 3*x**2 + 3*x + 3, x, domain='ZZ')
f(x) * g(x) = Poly(3*x**5 + x**4 + 8*x**3 + 5*x**2 + 5*x + 2, x, domain='ZZ')


Bedeutung von 'Domain':
- 'ZZ' Ganze Zahlen 
- 'QQ' Rationale Zahlen 
- 'RR' Reelle Zahlen 
- 'CC' Komplexe Zahlen 
- GF(p) Endlicher Körper z.B. GF(3329) für Rechnen mod 3329 (wie in Kyber)


# Was bedeutet `domain=GF(q)` in einem Polynom?

Wenn wir in Python (z. B. mit `sympy`) ein Polynom mit `domain=GF(q)` definieren, sagen wir:

*"Alle Koeffizienten des Polynoms liegen in einem **endlichen Körper** \( \mathbb{Z}_q \), also werden alle Rechnungen **modulo \( q \)** durchgeführt."*

---

### Was ist ein endlicher Körper?

Ein **endlicher Körper** (engl. *finite field*) ist eine Menge mit endlich vielen Zahlen, in der man:
- **addieren**,  
- **subtrahieren**,  
- **multiplizieren**  
- und **dividieren** (außer durch 0) kann – **ohne Fehler oder Widersprüche**.

Er wird meist so geschrieben: 

$$
\text{GF}(q) = \mathbb{Z}_q
$$


wobei \( q \) eine **Primzahl** ist.

---

### Beispiel: GF(7) = Rechnen **modulo 7**

Das heißt: wir rechnen mit den Zahlen \( \{0, 1, 2, 3, 4, 5, 6\} \)

- \( 4 + 5 = 2 \) (denn \( 9 \mod 7 = 2 \))  
- \( 3 \times 3 = 2 \) (denn \( 9 \mod 7 = 2 \))  
- Jeder Wert \( \neq 0 \) hat ein **multiplikatives Inverses**

---

### Warum wichtig für Kryptographie?

Viele Post-Quantum-Verfahren wie **Kyber, Dilithium oder NTRU** verwenden Polynome, deren Koeffizienten in einem **endlichen Körper** liegen – z. B. 

$$
\( \mathbb{Z}_{3329} \)
$$

- Das ist effizient (kleiner Wertebereich)
- Mathematisch sicher (Körperstruktur)
- Modularrechnungen sind **leicht auf Hardware abbildbar**

---

### Fazit

> `domain=GF(q)` bedeutet:  
> Wir rechnen mit Polynomen, deren **Koeffizienten modulo \( q \)** verarbeitet werden – eine zentrale Technik in der Post-Quantum-Kryptographie.


# Modulare Berechnung von Mini-Kyber

In [3]:
from sympy import GF, div

# Wir arbeiten modulo 3329 – das ist der Primzahl-Modulus wie im Kyber-Kryptosystem
q = 3329  # Modulus wie in Kyber
x = symbols('x')

In [4]:
# Definiere zwei Polynome f(x) und g(x), deren Koeffizienten in Z_q liegen
# Das bedeutet: Alle Koeffizienten werden automatisch modulo 3329 reduziert
f = Poly(x**3 + 2*x + 1, x, domain=GF(q))
g = Poly(3*x**2 + x + 2, x, domain=GF(q))

In [ ]:
# Multipliziere die beiden Polynome
produkt = f * g

In [0]:
# Definiere das Reduktionspolynom x^4 + 1
# In Kyber arbeitet man modulo (x^n + 1), hier vereinfacht mit n=4
modulus = Poly(x**4 + 1, x, domain=GF(q))

In [5]:
# Berechne das Produkt modulo x^4 + 1 → entspricht der Arbeit in einem Polynomring
_, rest = div(produkt, modulus)
print("Reduktion modulo x^4 + 1:", rest)

Reduktion modulo x^4 + 1: Poly(8*x**3 + 5*x**2 + 2*x + 1, x, modulus=3329)


In [ ]:
#---
# Mini-Kyber-Simulation: Schlüsselgenerierung und Verschlüsselung
# Das folgende Beispiel zeigt, wie man mit Polynomen einen „geheimen Schlüssel“ und eine „Nachricht“ kombinieren könnte

# Geheimschlüssel s(x) – zufälliges Polynom (vereinfacht)
s = Poly(2*x**3 + x + 5, x, domain=GF(q))   # Geheim

In [ ]:
# Öffentliches Polynom a(x) – ähnlich wie in Kyber der öffentlich bekannte Parameter
a = Poly(123*x + 88, x, domain=GF(q))       # Öffentlich

In [ ]:
# Fehlerterm e(x) – simuliert den "Noise", der zur Sicherheit beiträgt
e = Poly(1, x, domain=GF(q))                # Rauschterm

In [ ]:
# "Verschlüsselung": p(x) = a(x) * s(x) + e(x)
p = a * s + e

In [6]:
# Reduziere die verschlüsselte Nachricht modulo (x^4 + 1), wie im Kyber-Ring
_, p_reduced = div(p, modulus)
print("Öffentliche Nachricht (verschlüsselt):", p_reduced)


Öffentliche Nachricht (verschlüsselt): Poly(176*x**3 + 123*x**2 + 703*x + 195, x, modulus=3329)


# Aufgabe

# Aufgabe: Textverschlüsselung mit Polynomen in \( \mathbb{Z}_{3329}[x]/(x^n + 1) \)

## Ziel
In dieser Aufgabe implementierst du eine vereinfachte Form von Verschlüsselung, wie sie z. B. im Post-Quantum-Kryptosystem **Kyber** vorkommt. Du wandelst Text in ein Polynom um, verschlüsselst ihn über Polynom-Arithmetik in einem Restklassenring und gewinnst die Nachricht zurück.

---

## Teilaufgaben

### 1. Klartext vorbereiten
- Wähle einen kurzen Text (z.B. `Hallo`, `Hi!`, `Test123`)
- Wandle den Text in eine Liste von **ASCII-Zahlen** um
  - Beispiel: `"Hi!" → [72, 105, 33]`

### 2. Nachrichtspolynom erzeugen
- Erzeuge aus der Zahlencodierung ein **Polynom**:
  - $$\( m(x) = a_0 + a_1x + a_2x^2 + \dots \)$$
- Rechne im Ring $$\( \mathbb{Z}_{3329}[x]/(x^n + 1) \)$$ mit z.B. $$\( n = 8 \)$$
- Verwende `sympy.Poly(..., domain=GF(3329))`

### 3. Schlüssel und Verschlüsselung
- Definiere ein **öffentliches Polynom** $$\( a(x) \)$$, z.B. zufällig oder manuell mit Grad ≤ 3
- Definiere ein **geheimes Polynom** $$\( s(x) \)$$
- Optional: Erzeuge ein kleines „Rauschpolynom“ $$\( e(x) \)$$ (z.B. mit kleinen Zufallswerten)

- **Verschlüssele** mit:
$$
  \[
  c(x) = a(x) \cdot s(x) + m(x) + e(x)
  \]
$$
- **Reduziere das Ergebnis** modulo $$\( x^n + 1 \)$$, z. B. mit `sympy.div(...)`

### 4.  Entschlüsselung
- Berechne $$\( \hat{m}(x) = c(x) - a(x) \cdot s(x) \)$$
- Reduziere erneut modulo $$\( x^n + 1 \)$$
- Konvertiere die Koeffizienten von $$\( \hat{m}(x) \)$$ zurück in Zeichen (`chr()`)

---

## Hinweise & Hilfsmittel

- Du arbeitest über dem endlichen Körper  $$\( \mathbb{Z}_{3329} \)$$  
  → Verwende `GF(3329)` in `sympy.Poly(...)`
- Reduktion modulo $$\( x^n + 1 \)$$:  
  → `div(p, modulus)[1]` gibt dir den Rest (den reduzierten Teil)
- `ord(c)` konvertiert Buchstaben zu ASCII, `chr(n)` zurück

---

## Bonus-Aufgaben (optional)

- Implementiere eine kleine Funktion:
  ```python
  def encrypt(message: str, a, s, modulus, q=3329):
      ...